In [1]:
import sys
sys.path.append("../")
import datetime as dt
import random
import pandas as pd
import numpy as np
import util as ut
import random
import BagLearner as bal
import RTLearner as rtl
from indicators import bollingerbands, PriceSMA, MACD
from datetime import datetime, timedelta
from util import get_data

In [2]:
class StrategyLearner(object):

    def __init__(self, verbose=False, impact=0.0, commission=0.0):
        """  		  	   		  		 			  		 			 	 	 		 		 	
        Constructor method  		  	   		  		 			  		 			 	 	 		 		 	
        """
        self.verbose = verbose
        self.impact = impact
        self.commission = commission
        self.learner = bal.BagLearner(learner=rtl.RTLearner, verbose=False, kwargs={
            "leaf_size": 5}, bags=40, boost=False)

    def calculate_yval(self,price):
        return (price.shift(-10) - price) / price

    def calculate_mean_std(self,yval):
        # Calculate mean and StdDev of yval
        mean = yval.mean()
        std = yval.std()
        return mean, std

    def calculate_buy_limit_sell_limit(self,mean, std):
        # Calculate buy_limit and sell_limit based on mean and StdDev
        buy_limit = mean + std * 0.5
        sell_limit = mean - std * 0.5
        return buy_limit, sell_limit

    def create_train_dataframe(self,price, sd, ed, symbol, lookback, impact):
        yval = self.calculate_yval(price)
        train = pd.concat([
            price,
            bollingerbands(sd, ed, symbol, window_size=lookback),
            MACD(sd, ed, symbol),
            PriceSMA(sd, ed, symbol, window_size=lookback),
            yval
        ], axis=1)
        train.columns = [symbol, 'bbp', 'MACD_Ratio', 'PriceOverSMA', 'Yval']
        mean, std = self.calculate_mean_std(yval)
        buy_limit, sell_limit = self.calculate_buy_limit_sell_limit(mean, std)
        train['Y'] = np.select([
            (train['Yval'] - impact > buy_limit),
            (train['Yval'] + impact < sell_limit)
        ], [1, -1], default=0)
        train.dropna(axis=0, how='any', inplace=True)
        return train.iloc[:, 0:-2].values, train.iloc[:, -1].values

    def add_evidence(
        self,
        symbol="IBM",
        sd=dt.datetime(2008, 1, 1),
        ed=dt.datetime(2009, 1, 1),
        sv=10000,
    ):
        lookback = 20
        new_sd = sd - dt.timedelta(days=lookback+15)
        df = get_data([symbol], pd.date_range(new_sd, ed))
        price = df[symbol]
        price = price.loc[sd:ed]

        # print price

        train = pd.concat(
            [price,
             bollingerbands(sd, ed, symbol, window_size=lookback),
             MACD(sd, ed, symbol),
             PriceSMA(sd, ed, symbol, window_size=lookback),
             self.calculate_yval(price)], axis=1)
        trainX, trainY = self.create_train_dataframe(price, sd, ed, symbol, lookback, self.impact)

        self.learner.add_evidence(trainX, trainY)

    def testPolicy(self, symbol="IBM",
                   sd=dt.datetime(2009, 1, 1),
                   ed=dt.datetime(2010, 1, 1),
                   sv=10000):

        lookback = 20
        sym = [symbol]
        new_sd = sd - timedelta(days=lookback+15)
        df = get_data([symbol], pd.date_range(new_sd, ed))
        price = df[symbol]
        price = price.loc[sd:ed]
        testX = pd.concat([price,
                           bollingerbands(
                               sd, ed, symbol, window_size=lookback),
                           MACD(sd, ed, symbol),
                           PriceSMA(sd, ed, symbol, window_size=lookback)], axis=1)
        testX.columns = [symbol, 'bbp', 'MACD_Ratio', 'PriceOverSMA']

        testX = testX.values

        queryY = self.learner.query(testX)
        queryQ = self.learner.query1(testX)

        # print("Y:\n",queryY[0])
        # print("Q:\n",queryQ)

        date2 = pd.date_range(sd, ed)
        df2 = ut.get_data(sym, date2)
        df_dates = df2[sym].loc[sd:ed]
        d = df_dates.shape[0] - 1

        trades = df_dates.copy()

        trades.ix[:] = 0.0
        holding = 0

        for i in range(0, d):
            if queryY[0][i] == 1.0:
                if holding == 0:
                    trades.ix[i, symbol] = 1000
                    holding = holding + 1000

                elif holding == 1000:
                    trades.ix[i, symbol] = 0

                elif holding == -1000:
                    trades.ix[i, symbol] = 2000
                    holding = holding + 2000

            elif queryY[0][i] == -1.0:
                if holding == 0:
                    trades.ix[i, symbol] = -1000
                    holding = holding - 1000

                elif holding == -1000:
                    trades.ix[i, symbol] = 0

                elif holding == 1000:
                    trades.ix[i, symbol] = -2000
                    holding = holding - 2000

            elif queryY[0][i] == 0.0:
                if holding == 1000:
                    trades.ix[i, symbol] = -1000
                    holding = holding - 1000

                elif holding == 0:
                    trades.ix[i, symbol] = 0

                elif holding == -1000:
                    trades.ix[i, symbol] = 1000
                    holding = holding + 1000

        return trades


In [3]:
sym = 'JPM'
start_dt = dt.datetime(2008, 1, 1)
end_dt = dt.datetime(2009, 12, 31)
startval = 100000
stl = StrategyLearner(verbose=False, impact=0.0)
stl.add_evidence(sym, start_dt, end_dt, startval)
stl.testPolicy(sym, start_dt, end_dt, startval)

V:
 [[-1  0  0  0  1  1  0  0  1 -1  1  1 -1  1  0  1  0  0 -1  0 -1 -1  1  1
   0  0  0  0  0  0  0  1 -1 -1 -1  0  0  0  0 -1 -1 -1  1 -1  0  0  1 -1
  -1  1  1  1  1  0 -1  0 -1  0  0  0  0 -1  0  0 -1  1  0 -1 -1  1  1  1
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  1  0  1  0 -1 -1
   1  1 -1 -1 -1 -1  0 -1 -1 -1 -1  1 -1  0  0  1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1  0  0  1  1  1  0  1 -1 -1  0 -1  0  0  1  0  0 -1  0 -1  0
   0  0  0 -1 -1  0  1 -1 -1 -1  0 -1  0  0 -1  1  0  1 -1 -1  0  1  0  0
   0 -1  0  0 -1  0  0  0  1 -1  1  0  1  0  1  0  0 -1 -1  0 -1 -1 -1  0
  -1  0 -1  1 -1 -1 -1 -1 -1  0  0  0  0  0  0 -1  0  0 -1  0  0 -1 -1 -1
   0  0 -1 -1 -1 -1 -1  1  1  1  1  1  1  0 -1  1  1 -1  0  0 -1 -1 -1  0
   0  0  0  0  0  0  0 -1 -1 -1 -1  0  0  0 -1  1 -1  1  1  0  0  0  0 -1
   1 -1  0  1  1  0 -1  0  0  1 -1 -1  0 -1 -1  0  0  0  0  0  1 -1 -1 -1
   1 -1  1  0 -1 -1  0  1  1  1 -1  1 -1  1  0  0  0  1  1  0 -1  0  0 -1
   1  1  1  1 -1  0 -1  1  0  0  0

/home/devansh/anaconda3/envs/ml4t/lib/python3.6/site-packages/ipykernel_launcher.py:105: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/devansh/anaconda3/envs/ml4t/lib/python3.6/site-packages/ipykernel_launcher.py:123: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/devansh/anaconda3/envs/ml4t/lib/python3.6/site-packages/ipykernel_launcher.py:142: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/home/devansh/anaconda3/envs/ml4t/lib/python3.6/site-pa

,JPM
2008-01-02,-1000.0
2008-01-03,1000.0
2008-01-04,0.0
2008-01-07,0.0
2008-01-08,1000.0
2008-01-09,0.0
2008-01-10,-1000.0
2008-01-11,0.0
2008-01-14,1000.0
2008-01-15,-2000.0
